In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

Using TensorFlow backend.


In [2]:
IMG_SIZE = 128

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=True,
                                  zoom_range=.15,
                                  validation_split=.2,
                                   width_shift_range=.1,
                                   height_shift_range=.1
                                  )

train_generator = train_datagen.flow_from_directory(r'./train',
                                                   target_size=(IMG_SIZE,IMG_SIZE),
                                                   class_mode='binary',
                                                   subset='training',
                                                   )

validation_generator = train_datagen.flow_from_directory(r'./train',
                                                        target_size=(IMG_SIZE,IMG_SIZE),
                                                        class_mode='binary',
                                                        subset='validation')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
model = Sequential()

model.add(Conv2D(128, input_shape=(IMG_SIZE,IMG_SIZE,3), kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
model.fit_generator(train_generator, epochs=20, validation_data=validation_generator, use_multiprocessing=True)

Epoch 1/20
625/625 [==============================] - 162s 260ms/step - loss: 0.6825 - accuracy: 0.5566 - val_loss: 0.7592 - val_accuracy: 0.5504
Epoch 2/20
625/625 [==============================] - 153s 244ms/step - loss: 0.6437 - accuracy: 0.6287 - val_loss: 0.5408 - val_accuracy: 0.6852
Epoch 3/20
625/625 [==============================] - 156s 250ms/step - loss: 0.5893 - accuracy: 0.6817 - val_loss: 0.6282 - val_accuracy: 0.7182
Epoch 4/20
625/625 [==============================] - 161s 258ms/step - loss: 0.5526 - accuracy: 0.7146 - val_loss: 0.5190 - val_accuracy: 0.7442
Epoch 5/20
625/625 [==============================] - 154s 246ms/step - loss: 0.5105 - accuracy: 0.7511 - val_loss: 0.3634 - val_accuracy: 0.7686
Epoch 6/20
625/625 [==============================] - 154s 247ms/step - loss: 0.4627 - accuracy: 0.7792 - val_loss: 0.5688 - val_accuracy: 0.7590
Epoch 7/20
625/625 [==============================] - 154s 247ms/step - loss: 0.4121 - accuracy: 0.8110 - val_loss: 0.3159 -

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(r'./test',
                              target_size=(IMG_SIZE,IMG_SIZE),
                              shuffle=False)

Found 12500 images belonging to 1 classes.


In [6]:
predictions = model.predict_generator(test_generator, use_multiprocessing=True, verbose=1)

391/391 [==============================] - 31s 79ms/step


In [7]:
import pandas as pd

output = pd.DataFrame([[i+1, predictions[i][0]] for i in range(0, len(predictions))], columns=['id', 'label'])
output.to_csv('submission.csv', index=False)